In [1]:
! pip install datasets transformers rouge-score nltk

In [32]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [4]:
import transformers

print(transformers.__version__)

4.19.2


In [5]:
model_checkpoint = "sshleifer/distilbart-cnn-12-6"

In [6]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
from datasets import load_dataset, Dataset
import pandas as pd
df= pd.read_csv('My Drive/amazon_summary_data.csv')
df = df[['review_headline', 'review_body']]
df = df.sample(frac=0.05)
# df = df.sample()
# df.columns
df = df[['review_body','review_headline']]
df = df.dropna()
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [7]:
from datasets import load_dataset, load_metric

# raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")

In [8]:
dataset["train"][0]

{'__index_level_0__': 22362,
 'review_body': 'Terrible. Not sure if this is a fake wet brush but it repeatedly comes apart!',
 'review_headline': 'One Star'}

In [9]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(dataset["train"])

,review_body,review_headline,__index_level_0__
0,Goes on pretty easily with a light touch. Doesn't make the mess my former product did - and looks better too!,I'm sold:-),254592
1,great!,Five Stars,344001
2,Best mouth wash ever,Five Stars,335799
3,It's a good set of nail grooming tools. I like it.,Four Stars,11841
4,"So I am 15 years old and wanted a fresh sent that could be worn at dances and other party like atmospheres. I have never smelt this sent before and bought it off of YouTube reviews and other reviewers on Amazon. I bought the 100 ml bottle and was quite suprised at how big it was. THE SHIPPING TOOK ONLY 2 DAY!!!<br /><br />So I came across this fragrance choice when I typed in &#34;cologne that smells like vanilla and mint&#34;. I have seen reviews that say it smells like lemon... Well I smell no lemon in this cologne but rather a heavy mint with touches of vanilla. I have never smelled a cologne (out of my fathers, mine, and Dillard's, Macy's, ECT...) this is a unique smell that cools you. It reminds me of winter mint gum (not that it smells like gum but the cooling effect). It does smell fresh. This is defiantly more of a winter night cologne but could be worn in other seasons.<br /><br />The cologne is very strong; so much that the smell of the cologne you will notice after hours. It won't go full to your senses because you have became accustom to the smell. This cologne is a very manly smell to it while maintaining the sweet cooling smell. This is a smell I would say works best with age 14-28. Also it's projection is only a few feet with 1 spray; although it is VERY STRONG it doesn't project as good as it could. The longer you have it on the sweeter it gets. And it should last about 8 hours on skin. On cloths it will last days.<br /><br />The box is like a vault that holds gold at Fort Knox. The box is very difficult to open.<br /><br />O would recommend this cologne and I bought it directly from Amazon. It is the real deal- with a discount (about 30 dollars cheaper) YOU WONT BE DISAPPOINTED!!!",Blind buy,288664


In [11]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [12]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

In [13]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [14]:
tokenizer("Hello, this one sentence!")

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[0, 31414, 6, 42, 65, 3645, 328, 2], [0, 713, 16, 277, 3645, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


In [16]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [17]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["review_body"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["review_headline"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[0, 32579, 24786, 4, 1491, 686, 114, 42, 16, 10, 4486, 7727, 11822, 53, 24, 3987, 606, 4102, 328, 2], [0, 713, 16, 127, 78, 86, 7, 860, 42, 1152, 8, 38, 101, 24, 182, 203, 4, 1437, 38, 2740, 5, 2233, 9, 231, 7728, 25587, 6, 142, 24, 21, 5, 21084, 169, 7, 213, 4, 85, 21, 540, 1152, 87, 38, 5207, 13, 4, 125, 24, 202, 851, 162, 10, 778, 7, 192, 141, 127, 652, 74, 2519, 4, 85, 2334, 157, 328, 2860, 6, 11, 754, 328, 38, 437, 416, 21090, 7, 5494, 179, 1168, 6, 98, 89, 21, 117, 6574, 50, 932, 4, 38, 33, 341, 6710, 9856, 132, 1178, 13, 107, 4, 125, 14, 21, 27251, 6, 98, 854, 2547, 328, 41552, 3809, 48709, 100, 120, 7, 465, 10, 5010, 328, 36, 7608, 109, 51, 109, 42, 7, 201, 28749, 43, 41552, 3809, 48709, 45273, 347, 242, 46803, 18, 9944, 179, 1168, 321, 4, 245, 888, 1411, 15, 55, 17359, 87, 6710, 9856, 4, 20, 319, 1499, 2653, 11052, 4122, 4, 85, 156, 127, 3024, 619, 11052, 4122, 4, 38, 101, 24, 10, 319, 328, 41552, 3809, 48709, 100, 40, 2299, 907, 42, 1152, 456, 4, 13088, 6, 11, 

In [19]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/24 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [20]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [21]:
batch_size = 4
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [23]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [24]:
%cd /gdrive/MyDrive/

/gdrive/MyDrive


In [25]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/gdrive/MyDrive/distilbart-cnn-12-6-finetuned-xsum is already a clone of https://huggingface.co/sumedh/distilbart-cnn-12-6-finetuned-xsum. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend


In [26]:
%pip install wandb

In [29]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: __index_level_0__, review_headline, review_body. If __index_level_0__, review_headline, review_body are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 23014
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 5754


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.287500,1.629388,11.009000,7.461800,10.557300,10.808700,58.338200


Saving model checkpoint to distilbart-cnn-12-6-finetuned-xsum/checkpoint-500
Configuration saved in distilbart-cnn-12-6-finetuned-xsum/checkpoint-500/config.json
Model weights saved in distilbart-cnn-12-6-finetuned-xsum/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbart-cnn-12-6-finetuned-xsum/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbart-cnn-12-6-finetuned-xsum/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [distilbart-cnn-12-6-finetuned-xsum/checkpoint-4500] due to args.save_total_limit
Saving model checkpoint to distilbart-cnn-12-6-finetuned-xsum/checkpoint-1000
Configuration saved in distilbart-cnn-12-6-finetuned-xsum/checkpoint-1000/config.json
Model weights saved in distilbart-cnn-12-6-finetuned-xsum/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in distilbart-cnn-12-6-finetuned-xsum/checkpoint-1000/tokenizer_config.json
Special tokens file saved in distilbart-cnn-12-6-finetuned-xsum/checkpoin

Epoch,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5754, training_loss=1.2821295946722524, metrics={'train_runtime': 1745.4432, 'train_samples_per_second': 13.185, 'train_steps_per_second': 3.297, 'total_flos': 3899674265911296.0, 'train_loss': 1.2821295946722524, 'epoch': 1.0})